In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
import os

In [2]:

# Load pose estimation model (YOLOv8-pose)
model = YOLO("yolov8s-pose.pt")  # or yolov8x-pose.pt for more accuracy

# Open input video
video_path = "Video-2.mp4"
assert os.path.exists(video_path), "Video not found!"
cap = cv2.VideoCapture(video_path)

100%|██████████| 22.4M/22.4M [00:09<00:00, 2.49MB/s]


In [3]:

# Setup output video
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter("Video-2-PoseDetected.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))


In [ ]:

# Function to calculate angle at joint
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    angle = np.arccos(np.clip(np.dot((a - b), (c - b)) / (np.linalg.norm(a - b) * np.linalg.norm(c - b)), -1.0, 1.0))
    return np.degrees(angle)

In [ ]:


# Frame-by-frame analysis
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv8 pose estimation
    results = model(frame)

    # Annotate frame
    annotated = results[0].plot()

    # Extract keypoints (for first detected person)
    if results[0].keypoints is not None:
        kps = results[0].keypoints.xy[0].cpu().numpy()  # shape: (17, 2)

        # Get right shoulder, elbow, wrist
        right_shoulder = kps[6]
        right_elbow = kps[8]
        right_wrist = kps[10]

                # Ensure keypoints are valid (not near 0,0)
        if all(np.linalg.norm(p) > 5 for p in [right_shoulder, right_elbow, right_wrist]):
            angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

            if not np.isnan(angle):
                cv2.putText(annotated, f"Right Elbow: {int(angle)} deg",
                            (50, 50), cv2.FONT_HERSHEY_SIMPLEX,
                            1, (0, 255, 0), 2)



    out.write(annotated)

    cv2.imshow("Pose Detection", annotated)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print("✅ Video saved as 'Video-2-PoseDetected.mp4'")



0: 480x640 5 persons, 487.0ms
Speed: 9.2ms preprocess, 487.0ms inference, 4.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 persons, 483.2ms
Speed: 6.4ms preprocess, 483.2ms inference, 3.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 persons, 429.0ms
Speed: 6.5ms preprocess, 429.0ms inference, 3.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 persons, 446.0ms
Speed: 5.0ms preprocess, 446.0ms inference, 4.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 persons, 480.2ms
Speed: 7.6ms preprocess, 480.2ms inference, 4.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 persons, 458.0ms
Speed: 7.7ms preprocess, 458.0ms inference, 3.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 persons, 488.8ms
Speed: 6.4ms preprocess, 488.8ms inference, 3.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 6 persons, 511.8ms
Speed: 9.8ms preprocess, 511.8ms inference, 3.5ms postprocess per 